## fork from https://www.kaggle.com/code/shinya7y/great-barrier-reef-cascade-r-cnn-private-0-694

## Reference

* https://www.kaggle.com/mlneo07/mmdetection-swin-transfomer-frcnn-inference-0-443
* https://www.kaggle.com/c/tensorflow-great-barrier-reef/overview/evaluation
* https://www.kaggle.com/kocha1/only-yolov5-tracking-lb-642/notebook
* https://www.kaggle.com/parapapapam/yolox-inference-tracking-on-cots-lb-0-539

## Installation

In [ ]:
# !ls /kaggle/input/
# !ls /kaggle/input/universenet-for-offline/

In [ ]:
# install norfair and its dependencies
!pip install '/kaggle/input/norfair031py3/commonmark-0.9.1-py2.py3-none-any.whl' -f ./ --no-index
!pip install '/kaggle/input/norfair031py3/rich-9.13.0-py3-none-any.whl' --no-deps
!cp -r /kaggle/input/norfair031py3/filterpy-1.4.5/filterpy-1.4.5 /kaggle/working/
!pip install '/kaggle/working/filterpy-1.4.5'
!rm -rf '/kaggle/working/filterpy-1.4.5'
!pip install '/kaggle/input/norfair031py3/norfair-0.3.1-py3-none-any.whl' -f ./ --no-index --no-deps

In [ ]:
# downgrade pytorch to avoid compatibility issue with mmcv whl
!pip install '/kaggle/input/pytorch-190/torch-1.9.0+cu111-cp37-cp37m-linux_x86_64.whl'

In [ ]:
# install mmdet requirements
!pip install '/kaggle/input/universenet-for-offline/addict-2.4.0-py3-none-any.whl' --no-deps
!pip install '/kaggle/input/universenet-for-offline/yapf-0.32.0-py2.py3-none-any.whl' --no-deps
!pip install '/kaggle/input/universenet-for-offline/terminaltables-3.1.10-py2.py3-none-any.whl' --no-deps
!pip install '/kaggle/input/universenet-for-offline/mmcv_full-1.4.4-cp37-cp37m-manylinux1_x86_64.whl' --no-deps
!cp -r '/kaggle/input/universenet-for-offline/pycocotools-2.0.2/pycocotools-2.0.2' /kaggle/working/
!pip install '/kaggle/working/pycocotools-2.0.2' --no-deps
!rm -rf '/kaggle/working/pycocotools-2.0.2'

In [ ]:
# install UniverseNet
!cp -r '/kaggle/input/universenet-for-offline/UniverseNet-master' /kaggle/working/UniverseNet
!pip install -e '/kaggle/working/UniverseNet'

## Helper

In [ ]:
import numpy as np
from norfair import Detection, Tracker

def to_norfair(detects, frame_id):
    """Convert bbox format from xyxy+score to norfair.Detection class."""
    result = []
    for x_min, y_min, x_max, y_max, score in detects:
        xc, yc = (x_min + x_max) / 2, (y_min + y_max) / 2
        w, h = x_max - x_min, y_max - y_min
        result.append(
            Detection(
                points=np.array([xc, yc]),
                scores=np.array([score]),
                data=np.array([w, h, frame_id])))
    return result

def euclidean_distance(detection, tracked_object):
    """Euclidean distance function for norfair tracker."""
    # match detections on this frame with tracked_objects from previous frames
    return np.linalg.norm(detection.points - tracked_object.estimate)

In [ ]:
def result_to_gbr_str(result):
    """Convert mmdet result to GBR competition format."""
    result = result[0].copy()  # cots class only
    bboxes = result[:, :4]
    scores = result[:, 4]
    # xyxy2xywh
    bboxes[:, 2] = bboxes[:, 2] - bboxes[:, 0]  # width
    bboxes[:, 3] = bboxes[:, 3] - bboxes[:, 1]  # height

    pred_strings = []
    for box, score in zip(bboxes, scores):
        x_min, y_min, bbox_width, bbox_height = box
        pred_strings.append(f'{score:.4f} {x_min} {y_min} {bbox_width} {bbox_height}')
    gbr_str = ' '.join(pred_strings)
    return gbr_str

def result_to_gbr_str_with_tracker(result, tracker, frame_id):
    """Convert mmdet result and tracked result to GBR competition format.

    TODO: refactor to split tracking
    """
    result = result[0].copy()  # cots class only
    bboxes = result[:, :4]
    scores = result[:, 4]
    # xyxy2xywh
    bboxes[:, 2] = bboxes[:, 2] - bboxes[:, 0]  # width
    bboxes[:, 3] = bboxes[:, 3] - bboxes[:, 1]  # height

    detects = []
    pred_strings = []

    for box, score in zip(bboxes, scores):
        x_min, y_min, bbox_width, bbox_height = box
        detects.append([x_min, y_min, x_min+bbox_width, y_min+bbox_height, score])
        pred_strings.append(f'{score:.4f} {x_min} {y_min} {bbox_width} {bbox_height}')
    # Update tracks using detects from current frame
    tracked_objects = tracker.update(detections=to_norfair(detects, frame_id))

    for tobj in tracked_objects:
        bbox_width, bbox_height, last_detected_frame_id = tobj.last_detection.data
        if last_detected_frame_id == frame_id:  # Skip objects that were detected on current frame
            continue
        # Add objects that have no detections on current frame to predictions
        xc, yc = tobj.estimate[0]
        x_min = xc - bbox_width / 2
        y_min = yc - bbox_height / 2
        score = tobj.last_detection.scores[0]
        pred_strings.append(f'{score:.4f} {x_min} {y_min} {bbox_width} {bbox_height}')

    gbr_str = ' '.join(pred_strings)
    return gbr_str

## Inference

In [ ]:
import sys
sys.path.append('./UniverseNet')
sys.path.append('../input/tensorflow-great-barrier-reef/greatbarrierreef')
import cv2
import torch
from mmcv import Config
from mmdet.apis import init_detector, inference_detector, show_result_pyplot
import greatbarrierreef

In [ ]:
# call env.iter_test() before loading model to reduce risk of out of memory
env = greatbarrierreef.make_env()
iter_test = env.iter_test()

In [ ]:
#######################################################################################################
def prepare_mmdet_cots_model():
    device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
    checkpoint = '../input/gbr-starfish/cascade_rcnn_r2_101_fpn_fp16_4x2_mixup_affine_hsv_1440_lr002_7e_cots_alltrain_1600pafpn.pth'
    config_path = '../input/gbr-starfish/cascade_rcnn_r2_101_fpn_fp16_4x2_mixup_affine_hsv_1440_lr002_7e_cots_alltrain_1600pafpn.py'
    config = Config.fromfile(config_path)

    # TODO check settings
    config.model.test_cfg.rcnn.score_thr = 0.3
    print(config.model.test_cfg.rcnn)
    config.data.test.pipeline[1].img_scale = (2560, 1440)
    config.data.test.pipeline[1].flip = True
    print(config.data.test.pipeline[1])
    # TODO check speed and accuracy with fp16 on P100
    # config.fp16 = dict(loss_scale=dict(init_scale=512))
    config.fp16 = None

    model = init_detector(config, checkpoint, device=device)
    return model

model = prepare_mmdet_cots_model()
#######################################################################################################

In [ ]:
def infer_gbr_train(model):
    """Infer GBR train data."""
    tracker = Tracker(
        distance_function=euclidean_distance, 
        distance_threshold=30,
        hit_inertia_min=2,
        hit_inertia_max=4,
        initialization_delay=1)

    image_paths = ['/kaggle/input/tensorflow-great-barrier-reef/train_images/video_1/9114.jpg']
    image_paths += ['/kaggle/input/tensorflow-great-barrier-reef/train_images/video_2/5766.jpg']
#     image_paths += ['/kaggle/working/UniverseNet/demo/demo.jpg']
    for image_index, image_path in enumerate(image_paths):
        frame_id = image_index
        image_bgr = cv2.imread(image_path)
        result = inference_detector(model, image_bgr)
        gbr_str = result_to_gbr_str_with_tracker(result, tracker, frame_id)
        print(gbr_str)
        show_result_pyplot(model, image_bgr, result, score_thr=0., palette=(255, 111, 0))

infer_gbr_train(model)

In [ ]:
def infer_gbr_test(model):
    """Infer GBR test data using time-series API."""
    tracker = Tracker(
        distance_function=euclidean_distance, 
        distance_threshold=30,
        hit_inertia_min=2,
        hit_inertia_max=4,
        initialization_delay=1)

    for image_index, (image_rgb, pred_df) in enumerate(iter_test):
        frame_id = image_index
        image_bgr = image_rgb[:, :, ::-1]
        result = inference_detector(model, image_bgr)
        gbr_str = result_to_gbr_str_with_tracker(result, tracker, frame_id)
        pred_df['annotations'] = gbr_str
        env.predict(pred_df)
        if image_index < 3:
            print(gbr_str)
            show_result_pyplot(model, image_bgr, result, score_thr=0., palette=(255, 111, 0))

infer_gbr_test(model)

In [ ]:
!rm -rf '/kaggle/working/UniverseNet'